<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/Miky_HarumiSushi_yelp_picture_downloaderRandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#da cambiare:
# folder_id 
import requests
from lxml import html
import shutil
import os
import json

def downloadUser(user_id, counter):

    #Creare la cartella sul Drive dove andranno le foto (in data > face_data > Foto_User > (nomeRistorante)10%(strategia))
    folder_id = '1EdhoU-1Kqj8Ztf19NanbEYGK2UA402lo'  # folder ID = CaboFishTaco_10%Random
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id

    try:
        # Find user picture from web page
        page_content = requests.get(url)
        tree = html.fromstring(page_content.content)
        if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) != 0:  # Picture found
            image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

            # View all folders and file in your folder
            fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()

            # Create file on Google Drive
            # param id = ID of Fairness/face_data folder in my Google Drive
            drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})

            # Check if file already exists in Google Drive (prevents duplicates)
            for file in fileList:
                # print('Title: %s, ID: %s' % (file['title'], file['id']))
                if file['title'] == filename:  # The file already exists, then overwrite it
                    fileID = file['id']
                    drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})

            response = requests.get(image_url, stream=True)

            # Create a local copy of user picture
            with open(filename, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            del response

            # Upload user picture on Google Drive
            drive_file.SetContentFile(filename)  # path of local file content
            drive_file.Upload()  # Upload the file.
            # print('title: %s, mimeType: %s' % (drive_file['title'], drive_file['mimeType']))

            # Delete local user pictures
            if os.path.exists(filename):
                os.remove(filename)
                return False
            else:
                print("The file does not exist")
                return True

            return True
    except:
        print("Si è verificato un errore alla riga " + str(counter) + " del file user.json")
        return False

In [0]:
#Da cambiare:
# users_restaurant_id, il nome del csv e quante foto prendere (10% del totale delle recensioni)
# file_list mettere l'id della cartella da dove prendere le foto

#10% foto di utenti random per ristorante
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
import json
import csv
import random
import pandas as pd

# Connect to Google Account
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

nomeRistorante = 'Harumi Sushi'
# DOWNLOAD Restaurant.csv FROM DRIVE
# users_restaurant_id = id del file csv del ristorante (si trova in data > Scraping > Reviews_Relevance > (nomeRistorante)_relevance_desc.csv)
users_restaurant_id = '1EQJguMxV5AlnI21S_XIGKVwZHYf3bs0j'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': users_restaurant_id})
download.GetContentFile('%s' % nomeRistorante + '_date_desc.csv')

#leggo il csv
restaurant = pd.read_csv('%s' % nomeRistorante + '_date_desc.csv')

file_list = drive.ListFile({'q': "'1EdhoU-1Kqj8Ztf19NanbEYGK2UA402lo' in parents and trashed=false"}).GetList()
listaVecchia = []
for file1 in file_list:
  id = file1['title']
  listaVecchia.append(id[0:22])

#user_list = lista id utenti che ci sono anche nel dataset
user_list = []
for riga in restaurant['user_id']:
  if riga not in listaVecchia:
    user_list.append(riga)


#random_user = lista id utenti random (10% del totale)
#random_user = random.choices(user_list, k=300)
#cnt = 0
#Passo ogni id di random_user in downloadUser per scaricare le foto degli utenti
#for user_id in random_user:
  #id = '%s' % user_id
  #downloadUser(id,cnt)
  #print(cnt)
  #cnt = cnt + 1

In [6]:
#random_user = lista id utenti random (10% del totale)
#random_user = random.choices(user_list, k=50)

cnt = 0
foto_elaborate = 0
#Passo ogni id di random_user in downloadUser per scaricare le foto degli utenti
for user_id in user_list:
  id = '%s' % user_id
  if downloadUser(id,cnt):
      foto_elaborate = foto_elaborate + 1
  print(cnt)
  cnt = cnt + 1

print(foto_elaborate)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [1]:
foto_drive = drive.ListFile({'q': "'1EdhoU-1Kqj8Ztf19NanbEYGK2UA402lo' in parents and trashed=false"}).GetList()
lista_utenti_drive = []
for file1 in foto_drive:
  id = file1['title']
  lista_utenti_drive.append(id[0:22])

seen = set()
duplicati = []
for x in listaVecchia:
    if x not in seen:
      seen.add(x)
    else:
      duplicati.append(x)

print(len(listaVecchia))
print(duplicati)  

NameError: ignored